In [1]:
from neo4j import GraphDatabase

### Connection to Neo4j KG database

In [2]:
# Neo4j connection details
NEO4J_URI = "neo4j+s://fcc22d75.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "Abdo3VjaAxAfAZkkSWvRsQZN_IJ4EWTqUP9RTg0VwIc"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

print("Connected to Neo4j")

Connected to Neo4j


### Data exploration

Basic data exploration of the synthetic knowledge graph

In [3]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

In [5]:
# 1. Count number of nodes and relationships
node_count = run_query("MATCH (n) RETURN count(n) AS total_nodes")
relationship_count = run_query("MATCH ()-[r]->() RETURN count(r) AS total_relationships")
print("Total Nodes:", node_count[0]['total_nodes'])
print("Total Relationships:", relationship_count[0]['total_relationships'])

Total Nodes: 22845
Total Relationships: 63131


In [8]:
# 2. Get distinct node labels
node_labels = run_query("CALL db.labels() YIELD label RETURN label")
print("Node Labels:", [record['label'] for record in node_labels])
# 3. Get distinct relationship types
relationship_types = run_query("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType")
print("Relationship Types:", [record['relationshipType'] for record in relationship_types])

Node Labels: ['Document', 'Chunk', '__Entity__', 'Concept', 'Purpose', 'Behavior', 'Characteristic', 'Entity', 'Field', 'Person', 'Set', 'Strategy', 'Outcome', 'Protocol', 'Profession', 'Company', 'Product', 'Date', 'Standard', 'Domain', 'StockExchange', 'Location', 'UserBase', 'Platform', 'Patent', 'Title', 'Industry', 'Region', 'Feature', 'Stock Exchange', 'Organization', 'Software', 'Intellectual Property', 'Event', 'Service', 'Technology', 'Name', 'Market', 'Operating System', 'Decision', 'LegalDocument', 'Legal Decision', 'Monetary Value', 'Legal', 'Year', 'Authors', 'Video Game', 'Book', 'Course', 'Topic', 'Risk', 'Situation', 'Group', 'Effect', 'Hazard', 'Impact', 'Focus', 'Concern', 'Requirement', 'Role', 'Cause', 'Job Sector', 'Function', 'Action', 'Task', 'Input', 'Process', 'Goal', 'Problem', 'Data', 'Format', 'Issue', 'Reason', 'Condition', 'Context', 'Challenge', 'Control Method', 'Robot', 'Alias', 'Practice', 'Control', 'Measure', 'Program', 'Guideline', 'Framework', 'Pha

In [13]:
# 4. Get top 5 nodes with most relationships
top_nodes = run_query("MATCH (n)-[r]-() RETURN n, count(r) AS degree ORDER BY degree DESC LIMIT 5")
print("Top 5 Most Connected Nodes:")
for record in top_nodes:
    print(record['n'])

Top 5 Most Connected Nodes:
<Node element_id='4:f1d35087-bbee-462c-88fd-a0645044e884:465' labels=frozenset({'Product', 'Component', 'Label', 'Technology', 'Organization', 'Topic', 'Alias', 'Concept', 'Field', '__Entity__', 'Company', 'Entity'}) properties={'embedding': [-0.03653930127620697, -0.015164419077336788, 0.0164325051009655, 0.010568829253315926, 0.006010596640408039, -0.01847323589026928, 0.08546523749828339, 0.020968442782759666, 0.027815351262688637, 0.012431698851287365, -0.029376493766903877, -0.031135231256484985, 0.03491255268454552, -0.018150825053453445, -0.06498480588197708, 0.051682550460100174, -0.019606195390224457, -0.01573418639600277, -0.13371671736240387, -0.09645996987819672, -0.02547174133360386, -0.0014895638450980186, -0.0063493563793599606, -0.025820689275860786, -0.02737175114452839, 0.12268997728824615, -0.007792450953274965, -0.03852277994155884, 0.014383523724973202, -0.09218423068523407, 0.008695743046700954, 0.0026133842766284943, 0.0910346657037735

In [25]:
def get_textual_content(node_id):
    with driver.session() as session:
        query = """
        MATCH (n)
        WHERE n.id = $node_id
        RETURN n.text  // Replace 'text' with the correct textual property name
        """
        result = session.run(query, node_id=node_id)
        node = result.single()  # Returns the first match
        
        if node:
            return node['n.text']  # The textual content of the node
        else:
            return None

# Get the textual content of the node 
textual_content = get_textual_content('AI')
print(textual_content)

None


In [ ]:
def get_longest_textual_node():
    with driver.session() as session:
        query = """
        MATCH (n)
        WHERE n.text IS NOT NULL  // Check if the 'text' property is not null
        WITH n, size(n.text) AS text_length  // Calculate the length of 'text'
        ORDER BY text_length DESC  // Order nodes by text length in descending order
        LIMIT 1  // Get the node with the longest text
        RETURN n, text_length
        """
        result = session.run(query)
        node = result.single()  # Returns the first match (node with longest text)

        if node:
            return node['n'], node['text_length']  # Return node and its textual length
        else:
            return None

# Get the node with the longest textual content
longest_node, text_length = get_longest_textual_node()

# Handle the case where no node was found
if longest_node is not None:
    print(f"Node with the longest text: {longest_node}")
    print(f"Text length: {text_length}")
    print(f"Text: {longest_node['text']}")  
else:
    print("No nodes with textual content found.")

Node with the longest text: <Node element_id='4:f1d35087-bbee-462c-88fd-a0645044e884:19845' labels=frozenset({'Chunk'}) properties={'fileName': 'Algorithmic party platforms in the United States.txt', 'content_offset': 7156, 'length': 674, 'text': ' in political platforms also raises ethical and transparency concerns, particularly regarding the authenticity of dynamically adjusted messaging and the potential for voter manipulation. Addressing these challenges is crucial to maintaining voter trust and the integrity of the democratic process. In summary, AI significantly shapes political platforms in real time by providing campaigns with the tools to analyze voter sentiment, segment audiences, and adjust strategies dynamically. While offering substantial benefits in responsiveness and engagement, it is imperative to navigate the accompanying ethical considerations to ensure the responsible use of AI in political', 'embedding': [-0.026420066133141518, -0.04907726123929024, -0.0475012250244